# How to Build a Generative AI Model?
Building a generative AI model from scratch involves a dual-network setup consisting of a generator and a discriminator, unlike a typical machine learning model that usually involves a single predictive model.

In generative AI, the generator creates new data samples from random noise, while the discriminator evaluates these samples against real data to classify them as real or fake. The two networks are trained in tandem through an adversarial process where the generator aims to improve its ability to produce realistic outputs, and the discriminator enhances its accuracy in distinguishing between genuine and generated data.

In this article, I’ll take you through building a Generative AI model using Generative Adversarial Networks (GANs). So, let’s understand what GANs are.


Generative Adversarial Networks (GANs) consist of two neural networks:


Generator: Generates new data samples.
Discriminator: Evaluates whether a given data sample is real (from the training data) or fake (generated by the generator).

The two networks are trained together in a zero-sum game: the generator tries to fool the discriminator, while the discriminator aims to accurately distinguish real from fake data.

A GAN consists of the following key components:

Noise Vector: A random input vector fed into the generator.
Generator: A neural network that transforms the noise vector into a data sample.
Discriminator: A neural network that classifies input data as real or fake.

We will train a Generative AI model to generate images. Let’s import the necessary Python libraries and the dataset to get started with the task:

In [20]:
import numpy as np
import  matplotlib.pyplot as plt
from keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, InputLayer
from keras.layers import Input,Dense,Reshape,Flatten
from keras.models import Model
from keras.optimizers import Adam
import ssl
import urllib.request
import warnings
warnings.filterwarnings('ignore')

In [21]:
ssl._create_default_https_context = ssl._create_unverified_context

In [22]:
(X_train, _) , (_,_) = mnist.load_data()

Now, we will build a generator network. The generator network transforms a random noise vector into a data sample. We’ll use a simple feed-forward network with several layers:

In [23]:
def build_generator():
    model = Sequential([
        InputLayer(input_shape=(100,)),  # Proper input specification
        Dense(256),
        LeakyReLU(negative_slope=0.2),   # Updated parameter name
        BatchNormalization(momentum=0.8),
        Dense(512),
        LeakyReLU(negative_slope=0.2),
        BatchNormalization(momentum=0.8),
        Dense(1024),
        LeakyReLU(negative_slope=0.2),
        BatchNormalization(momentum=0.8),
        Dense(784, activation='tanh'),   # Output range [-1, 1] for GANs
        Reshape((28, 28, 1))            # MNIST-like image shape
    ])
    return model

# Build and verify
generator = build_generator()
generator.summary()  # Will show correct shapes/params

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 256)                 │          25,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_6 (LeakyReLU)            │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1024)                │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_7 (LeakyReLU)            │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 1024)                │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 784)                 │         803,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_1 (Reshape)                  │ (None, 28, 28, 1)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,493,520 (5.70 MB)

 Trainable params: 1,489,936 (5.68 MB)

 Non-trainable params: 3,584 (14.00 KB)

Our starts with a dense (fully connected) layer that takes a 100-dimensional noise vector as input. It passes through several layers, each comprising dense connections followed by a LeakyReLU activation function to introduce non-linearity, and BatchNormalization layers to stabilize training and improve convergence. The network then progressively increases the number of neurons, which culminates in a dense layer with 784 neurons, corresponding to the flattened pixel values of a 28×28 image.

This final output is reshaped to match the original image dimensions, with a ‘tanh’ activation function to output values in the range [-1, 1], suitable for image data. This setup enables the generator to transform random noise into structured image data that mimics the real data distribution.

Now, we will build the discriminator network. The discriminator network will classify input images as real or fake. It will be a binary classifier that outputs the probability of an image being real:

In [24]:
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(28,28,1)))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    return model

discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(0.0002,0.5), loss= 'binary_crossentropy',metrics=['accuracy'])

The discriminator network begins with a Flatten layer, which converts the 28×28 pixel images into a one-dimensional array to prepare it for fully connected layers. It then passes through two dense layers, with 512 and 256 neurons respectively, each followed by a LeakyReLU activation function to introduce non-linearity. The final dense layer has a single neuron with a ‘sigmoid’ activation, which outputs a probability score indicating whether the input image is real or fake.

The model is then compiled with the Adam optimizer and binary crossentropy loss function to optimize the network to accurately distinguish real images from those generated by the generator.The discriminator network begins with a Flatten layer, which converts the 28×28 pixel images into a one-dimensional array to prepare it for fully connected layers. It then passes through two dense layers, with 512 and 256 neurons respectively, each followed by a LeakyReLU activation function to introduce non-linearity. The final dense layer has a single neuron with a ‘sigmoid’ activation, which outputs a probability score indicating whether the input image is real or fake.

The model is then compiled with the Adam optimizer and binary crossentropy loss function to optimize the network to accurately distinguish real images from those generated by the generator.

# Compiling and Training the GAN to build a Generative AI Model to Generate Images
Now, we will combine the generator and discriminator to train our Generative AI model to generate images. During training, the discriminator will be trained to distinguish real from fake images, while the generator will be trained to produce images that fool the discriminator:

In [28]:
discriminator.trainable = False
gan_input = Input(shape=(100,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)

gan = Model(gan_input, gan_output)
gan.compile(optimizer = Adam(0.0002,0.5), loss = 'binary_crossentropy')

def train_gan(epochs,batch_size=128):
    X_train,_ = mnist.load_data()
    X_train = (X_train[0].astype(np.float32) - 127.5)/127.5
    X_train = np.expand_dims(X_train, axis = 3)
    real = np.ones((batch_size , 1))
    fake = np.zeros((batch_size,1))

    for epoch in range(epochs):
        idx = np.random.randint(0,X_train.shape[0], batch_size)
        real_images = X_train[idx]
        noise = np.random.normal(0,1,(batch_size,100))
        generated_images = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(real_images,real)
        d_loss_fake = discriminator.train_on_batch(generated_images,fake)
        d_loss = 0.5 * np.add(d_loss_real,d_loss_fake)
        noise = np.random.normal(0,1,(batch_size,100))
        g_loss = gan.train_on_batch(noise, real)
        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100* d_loss[1]} [G loss:{g_loss}]")
            save_images(epoch)

In [29]:
def save_images(epoch):
    r,c = 5,5
    noise = np.random.normal(0,1, (r * c, 100))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    fig,axs = plt.subplots(r,c)
    count = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(generated_images[count,:,:,0], cmap='gray')
            axs[i,j].axis('off')
            count += 1
    fig.savefig(f"gan_images_{epoch}.png")
    plt.close()

In [ ]:
train_gan(epochs= 10000, batch_size = 64)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0 [D loss: 5.170984268188477, acc.: 7.121262699365616 [G loss:9.553717063681688e-06]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2

n the above code, we are defining and training a Generative Adversarial Network by combining a generator and a discriminator into a single model.

The gan_input represents the random noise fed into the generator, which produces a generated_image. This image is then passed to the discriminator, which outputs a probability (gan_output) indicating whether the image is real or fake.

The discriminator’s weights are set to non-trainable during this process to ensure that only the generator learns from the adversarial feedback. The GAN is trained using the function train_gan, where the discriminator first learns to distinguish between real images and fake images generated by the generator, and then the generator is updated to produce more convincing fake images.

The loss functions guide this adversarial process, where the generator aims to minimize the discriminator’s ability to detect fakes, which results in progressively more realistic generated images. The save_images function periodically saves these generated images to visualize the training progress.

The model generated several images. Look at these three images in the output below:


n the above code, we are defining and training a Generative Adversarial Network by combining a generator and a discriminator into a single model.

The gan_input represents the random noise fed into the generator, which produces a generated_image. This image is then passed to the discriminator, which outputs a probability (gan_output) indicating whether the image is real or fake.

The discriminator’s weights are set to non-trainable during this process to ensure that only the generator learns from the adversarial feedback. The GAN is trained using the function train_gan, where the discriminator first learns to distinguish between real images and fake images generated by the generator, and then the generator is updated to produce more convincing fake images.

The loss functions guide this adversarial process, where the generator aims to minimize the discriminator’s ability to detect fakes, which results in progressively more realistic generated images. The save_images function periodically saves these generated images to visualize the training progress.

The model generated several images. Look at these three images in the output below:


n the above code, we are defining and training a Generative Adversarial Network by combining a generator and a discriminator into a single model.

The gan_input represents the random noise fed into the generator, which produces a generated_image. This image is then passed to the discriminator, which outputs a probability (gan_output) indicating whether the image is real or fake.

The discriminator’s weights are set to non-trainable during this process to ensure that only the generator learns from the adversarial feedback. The GAN is trained using the function train_gan, where the discriminator first learns to distinguish between real images and fake images generated by the generator, and then the generator is updated to produce more convincing fake images.

The loss functions guide this adversarial process, where the generator aims to minimize the discriminator’s ability to detect fakes, which results in progressively more realistic generated images. The save_images function periodically saves these generated images to visualize the training progress.

The model generated several images. Look at these three images in the output below:


